In [ ]:
%load_ext autoreload
%autoreload 2

import hyp
import utils
import os
import json
import pandas as pd
import numpy as np
from visualization import plot_circadian_cycle, plot_stacked_bar

In [ ]:
settings = json.load(open('./settings.json'))

path = settings['general']['dataset_dir']
subs = ['05', '24', '37']
tolerance = 45
hours_counts = []
counts = []

subs = utils.check_all_setting(subs, 'sub', path, prefix='sub')

for sub in subs:
    settings['general']['sub'] = sub
    settings['general']['ses'] = '34w'
    settings['general']['run'] = 'all'

    os.remove('./settings.json')
    with open('./settings.json', 'w') as f:
        json.dump(settings, f)

    sub = settings['general']['sub']
    ses = settings['general']['ses']

    path = os.path.join(settings['general']['dataset_dir'], settings['general']['eeg_subdir'].split('ses')[0].replace(
            '<SUB>', sub))

    ses = utils.check_all_setting(ses, 'ses', path, suffix='w')

    for _ses in ses:
        run = settings['general']['run']

        output_dir = utils.create_output_dir(settings['general']['output_dir'], sub, _ses)

        path = os.path.join(settings['general']['dataset_dir'], settings['general']['eeg_subdir'].replace(
            '<SUB>', sub).replace('<SES>', _ses))
        
        run = utils.check_all_setting(run, 'run', path, suffix=settings['hyp']['suffix'])

        dfs = []

        for _run in run:
            filename = utils.get_filename(settings, _ses, _run, settings['hyp']['suffix'])
            filename = os.path.join(path, filename)

            dfs.append(hyp.get_hyp_df(filename, settings))

        df = pd.concat(dfs, ignore_index=True)

        # FIXME:
        # Ma ha senso la normalizzazione? Ha senso solo troncare le ore monche sotto soglia, il padding introduce degli ignored che confliggono con il resto!
        # Alternativamente ha senso escludere dal denominatore delle percentuali gli ignored
        # df = hyp.normalize_dataframe(df, settings, tolerance)

        hours_counts.append(hyp.count_full_hours(df, settings, tolerance=45))
        quiet_counts = hyp.get_stage_cycle(df, settings, description='QuietSleep', tolerance=45)
        active_counts = hyp.get_stage_cycle(df, settings, description='ActiveSleep', tolerance=45)
        wake_counts = hyp.get_stage_cycle(df, settings, description='Wake', tolerance=45)
        counts.append([quiet_counts, active_counts, wake_counts])
        # hyp.count_adjacent_stages_per_hour(df, settings, description='ActiveSleep', min_duration=60)

hours_counts = np.sum(hours_counts, axis=0)

counts = np.sum(counts, axis=0)
counts = np.divide(counts, hours_counts, out=np.zeros_like(counts), where=hours_counts!=0)

In [ ]:
plot_circadian_cycle(counts, labels=('Quiet Sleep', 'Active Sleep','Wake'), color=['#9DD866', '#0B84A5','#CA472F'], title='Circadian Cycle (subs ' + str(subs) + ')')